------------------
```markdown
# Copyright © 2024 Meysam Goodarzi
This notebook is licensed under CC BY-NC 4.0 with the following amandments:
- Individuals may use, share, and adapt this material for non-commercial purposes with attribution.
- Institutions/Companies must obtain written consent to use this material, except for nonprofits.
- Commercial use is prohibited without permission.  
Contact: analytica@meysam-goodarzi.com
```
------------------------------
❗❗❗ **IMPORTANT**❗❗❗ **Create a copy of this notebook**

In order to work with this Google Colab you need to create a copy of it. Please **DO NOT** provide your answers here. Instead, work on the copy version. To make a copy:

**Click on: File -> save a copy in drive**

Have you successfully created the copy? if yes, there must be a new tab opened in your browser. Now move to the copy and start from there!

----------------------------------------------


# Gale-Shapely Algorithm
The Gale-Shapely algorithm solves the Stable Matching Problem, which aims to find a stable pairing between two groups based on preferences. It guarantees stability, ensures efficiency, and optimizes the outcome for the proposing side.
This algorithm is widely used in college admissions, job markets, and organ donation systems.

## College Admission Problem
The College Admission Problem involves:

- n students and n colleges.
- Each student ranks colleges in order of preference.
- Each college ranks students and has a fixed quota (maximum admissions).

The final goal is to find a stable matching where:
- No student-college pair prefers each other over their current match.
- Every student is matched to at most one college.
- Every college admits students up to its quota.

We could summarize the preferences the students as:

Students | Preferences |
 :- | -:- |
  $s_1$ | $c_1$ $> c_2$ $> c_3$
  $s_2$ | $c_2$ $> c_1$ $> c_3$  
  $s_3$ | $c_1$ $> c_3$ $> c_2$

and that of the colleges as:

Students | Preferences |
 :- | -:- |
  $c_1$ | $s_2$ $> s_1$ $> s_3$
  $c_2$ | $s_1$ $> s_3$ $>s_2$
  $c_3$ | $s_3$ $> s_2$ $> s_1$



### Step 1
we first need to define the preferences. We'll use dictionaries to store student and college preferences.

In [ ]:
# Students' preferences (ranked lists of colleges)
students_preferences = {
    "s1": ["c1", "c2", "c3"],
    "s2": ["c2", "c1", "c3"],
    "s3": ["c1", "c3", "c2"]
}

# Colleges' preferences (ranked lists of students)
colleges_preferences = {
    "c1": ["s2", "s1", "s3"],
    "c2": ["s1", "s3", "s2"],
    "c3": ["s3", "s2", "s1"]
}

### Step 2
We then create variables based on our scenario, i.e., who proposes to who.

```python
def proposers_proposee(self):
        """
        Determines proposers, proposees, and their preferences.

        Returns:
            tuple: (proposers, proposers_preferences,
                    proposees, proposees_preferences).
        """
        if self.proposing_group == 'A':
            proposers = self.group_A
            proposees = self.group_B
            proposers_preferences = self.A_preferences
            proposees_preferences = self.B_preferences
        elif self.proposing_group == 'B':
            proposers = self.group_B
            proposees = self.group_A
            proposers_preferences = self.B_preferences
            proposees_preferences = self.A_preferences
        else:
            raise ValueError("Proposing group must be 'A' or 'B'.")

        return proposers, proposers_preferences, proposees, proposees_preferences
```

### Step 3
Finally we implement the Gale-Shapley algorithm as follows:
1. Proposers propose to their top-choice proposees.
1. Proposees evaluate applicants and keep the most preferred proposers.
1. Rejected proposers move to the next preferred proposee.
1. Repeat until all proposers are either matched or exhausted all choices.

In [ ]:
class GaleShapley:
    def __init__(self, A_preferences, B_preferences, proposing_group="A"):
        """
        Initialize the Gale-Shapley algorithm.

        :param group_A: List of names/identifiers for the first group .
        :param group_B: List of names/identifiers for the second group.
        :param A_preferences: Dictionary where keys are members of group_A and values are their ranked preferences for group_B.
        :param B_preferences: Dictionary where keys are members of group_B and values are their ranked preferences for group_A.
        """
        self.group_A = list(A_preferences)
        self.group_B = list(B_preferences)
        self.A_preferences = A_preferences
        self.B_preferences = B_preferences
        self.proposing_group = proposing_group
        self.matches = {}  # Stores the final matches


    def proposers_proposee(self):
        """
        Determines proposers, proposees, and their preferences.

        Returns:
            tuple: (proposers, proposers_preferences,
                    proposees, proposees_preferences).
        """
        if self.proposing_group == 'A':
            proposers = self.group_A
            proposees = self.group_B
            proposers_preferences = self.A_preferences
            proposees_preferences = self.B_preferences
        elif self.proposing_group == 'B':
            proposers = self.group_B
            proposees = self.group_A
            proposers_preferences = self.B_preferences
            proposees_preferences = self.A_preferences
        else:
            raise ValueError("Proposing group must be 'A' or 'B'.")

        return proposers, proposers_preferences, proposees, proposees_preferences

    def match(self):
        """
        Run the Gale-Shapley algorithm to find a stable matching.

        Return:
          A dictionary representing the stable matching.
        """
        proposers, proposers_preferences, proposees, proposees_preferences = self.proposers_proposee()

        # Initialize all proposers as free and create a dictionary to track proposals
        free_proposers = list(proposers)
        proposals = {proposer: [] for proposer in proposers}  # Tracks proposals made by each proposer

        # While there are free proposers
        while free_proposers:
            proposer = free_proposers.pop(0)  # Pick a free proposer
            proposer_pref_list = proposers_preferences[proposer]  # Get their preference list

            # Propose to proposees in order of preference
            for proposee in proposer_pref_list:
                if proposee not in self.matches.values():  # If proposee is free, accept the proposal
                    self.matches[proposer] = proposee
                    break
                else:
                    # If proposee is already matched, check if they prefer the new proposer
                    current_match = [k for k, v in self.matches.items() if v == proposee][0]
                    proposee_pref_list = proposees_preferences[proposee]
                    if proposee_pref_list.index(proposer) < proposee_pref_list.index(current_match):
                        # Proposee prefers the new proposer, so reject the current match
                        self.matches[proposer] = proposee
                        free_proposers.append(current_match)  # The rejected proposer becomes free
                        break
                proposer_pref_list.remove(proposee)

        # Return the matches based on the proposing group
        return self.matches

### Testing the Algorithm
Now, we execute the function and print the results.

In [ ]:
# Create an instance of the GaleShapley class
gs = GaleShapley(students_preferences, colleges_preferences, proposing_group="A")

# Run the algorithm with Group A proposing to Group B
matches = gs.match()
print("Stable Matching (students proposing to colleges):", matches)

# Run the algorithm with Group B proposing to Group A
gs = GaleShapley(students_preferences, colleges_preferences, proposing_group="B")
matches = gs.match()
print("Stable Matching (colleges proposing to students):", matches)

## Exercise
We consider an organ transplant problem. We assume:

- n patients needing an organ transplant.
- n donors/hospitals offering organs.
- Each patient ranks donors based on compatibility & preference.
- Each donor ranks patients based on priority criteria.
- Each donor has a quota (number of transplants they can perform).

The goal is to find a stable matching where no unmatched patient-donor pair prefers each other over their current match.

We could summarize the preferences as:

Students | Preferences |
 :- | -:- |
  $p_1$ | $d_1$ $> d_2$ $> d_3$
  $p_2$ | $d_2$ $> d_1$ $> d_3$  
  $p_3$ | $d_1$ $> d_3$ $> d_2$

and that of the colleges as:

Students | Preferences |
 :- | -:- |
  $d_1$ | $p_2$ $> p_1$ $> p_3$
  $d_2$ | $p_1$ $> p_3$ $> p_2$
  $d_3$ | $p_3$ $> p_2$ $> p_1$

In [ ]:
# Your code

**Congratulations! You have finished the Notebook! Great Job!**
🤗🙌👍👏💪
<!--
# Copyright © 2024 Meysam Goodarzi
This notebook is licensed under CC BY-NC 4.0 with the following amandments:
- Individuals may use, share, and adapt this material for non-commercial purposes with attribution.
- Institutions/Companies must obtain written consent to use this material, except for nonprofits.
- Commercial use is prohibited without permission.  
Contact: analytica@meysam-goodarzi.com.
-->